In [2]:
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.nn import init, MarginRankingLoss
from transformers import BertModel, RobertaModel
from transformers import BertTokenizer, RobertaTokenizer
from torch.optim import Adam
from distutils.version import LooseVersion
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
from torch.autograd import Variable
import math
from transformers import AutoConfig, AutoModel, AutoTokenizer
import nltk
import re
import Levenshtein
import spacy
import en_core_web_sm
import torch.optim as optim
from tqdm import tqdm
from torch.distributions import Categorical
from numpy import linalg as LA
from transformers import AutoModelForMaskedLM
from nltk.corpus import wordnet
import torch.nn.functional as F
import random
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import precision_recall_fscore_support
from nltk.corpus import words as wal
from flair.data import Sentence
from flair.models import SequenceTagger

In [3]:
class MyDataset(Dataset):
    def __init__(self,file_name):
        df1 = pd.read_csv(file_name)
        df1 = df1.fillna("")
        res = df1['X']
        self.X_list = res
        self.y_list = df1['y']
    def __len__(self):
        return len(self.X_list)
    def __getitem__(self,idx):
        mapi = []
        mapi.append(self.X_list[idx])
        mapi.append(self.y_list[idx])
        return mapi

In [4]:
epoch_number = 0
EPOCHS = 5
run_int = 26
tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = AutoModelForMaskedLM.from_pretrained("microsoft/graphcodebert-base")
optimizer = Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=2e-3)
myDs=MyDataset('dat1.csv') # 10-20% of num_training_steps
model.train()
train_loader=DataLoader(myDs,batch_size=24,shuffle=True)
best_loss = torch.full((1,), fill_value=100000)

In [ ]:
for epoch in range(EPOCHS):
    loop = tqdm(train_loader, leave=True)
    tot_loss = 0.0
    for batch in loop:
        optimizer.zero_grad()
        inputs = batch
        maxi = torch.tensor(0.0, requires_grad=True)
        for i in range(len(batch[0])):
            X_init = inputs[0][i]
            y = inputs[1][i]
            ty = tokenizer.encode(y)[1:-1]
            print(y)
            nl = re.findall(r'[A-Z](?:[a-z]+|[A-Z]*(?=[A-Z]|$))|[a-z]+|\d+', y)
            lb = ' '.join(nl).lower()
            x = tokenizer.tokenize(lb)
            num_sub_tokens_label = len(x)
            X_init = X_init.replace("[MASK]", " ".join([tokenizer.mask_token] * num_sub_tokens_label))
            preds = []
            tokens = tokenizer.encode_plus(X_init, add_special_tokens=False,return_tensors='pt')
            input_id_chunki = tokens['input_ids'][0].split(510)
            input_id_chunks = []
            mask_chunks  = []
            mask_chunki = tokens['attention_mask'][0].split(510)
            for tensor in input_id_chunki:
                input_id_chunks.append(tensor)
            for tensor in mask_chunki:
                mask_chunks.append(tensor)
            xi = torch.full((1,), fill_value=101)
            yi = torch.full((1,), fill_value=1)
            zi = torch.full((1,), fill_value=102)
            for r in range(len(input_id_chunks)):
                input_id_chunks[r] = torch.cat([xi, input_id_chunks[r]],dim = -1)
                input_id_chunks[r] = torch.cat([input_id_chunks[r],zi],dim=-1)
                mask_chunks[r] = torch.cat([yi, mask_chunks[r]],dim=-1)
                mask_chunks[r] = torch.cat([mask_chunks[r],yi],dim=-1)
            di = torch.full((1,), fill_value=0)
            for i in range(len(input_id_chunks)):
                # get required padding length
                pad_len = 512 - input_id_chunks[i].shape[0]
                # check if tensor length satisfies required chunk size
                if pad_len > 0:
                    # if padding length is more than 0, we must add padding
                    for p in range(pad_len):
                        input_id_chunks[i] = torch.cat([input_id_chunks[i],di],dim=-1)
                        mask_chunks[i] = torch.cat([mask_chunks[i],di],dim=-1)
            input_ids = torch.stack(input_id_chunks)
            attention_mask = torch.stack(mask_chunks)
            maski = []
            u = 0
            ad = 0
            labels = input_ids.clone()
            for l in range(len(input_ids)):
                masked_pos = []
                for i in range(len(input_ids[l])):
                    if input_ids[l][i] == 50264: #103
                        u+=1
                        if i != 0 and input_ids[l][i-1] == 50264:
                            continue
                        for t in range(num_sub_tokens_label):
                            if (i+t) > len(labels[l]):
                                labels[l+1][i+t-len(labels[l])] = ty[t]
                            else:
                                labels[l][i+t] = ty[t]
                        masked_pos.append(i)
                        ad+=1
                    else:
                        labels[l][i] = -100
                maski.append(masked_pos)
            input_dict = {
                'input_ids': input_ids.long(),
                'attention_mask': attention_mask.int()
            }
            print('number of mask tok',u)
            print('number of seq', ad)
            outputs = model(input_dict['input_ids'], attention_mask = input_dict['attention_mask'])
            last_hidden_state = outputs[0].squeeze()
            l_o_l_sa = []
            sum_state = []
            for t in range(num_sub_tokens_label):
                c = []
                l_o_l_sa.append(c)
            if len(maski) == 1:
                masked_pos = maski[0]
                for k in masked_pos:
                    for t in range(num_sub_tokens_label):
                        l_o_l_sa[t].append(last_hidden_state[k+t])
            else:
                for p in range(len(maski)):
                    masked_pos = maski[p]
                    for k in masked_pos:
                        for t in range(num_sub_tokens_label):
                            if (k+t) > len(last_hidden_state[p]):
                                l_o_l_sa[t].append(last_hidden_state[p+1][k+t-len(last_hidden_state[p])])
                                continue
                            l_o_l_sa[t].append(last_hidden_state[p][k+t])
            for t in range(num_sub_tokens_label):
                sum_state.append(l_o_l_sa[t][0])
            for i in range(len(l_o_l_sa[0])):
                if i == 0:
                    continue
                for t in range(num_sub_tokens_label):
                    sum_state[t] = sum_state[t] + l_o_l_sa[t][i]
            yip = len(l_o_l_sa[0])
            qw = ""
            val = torch.tensor(0.0, requires_grad=True)
            for t in range(num_sub_tokens_label):
                sum_state[t] /= yip
                probs = F.softmax(sum_state[t], dim=0)
                val = val - torch.log(probs[ty[t]])
                idx = torch.topk(sum_state[t], k=1, dim=0)[1]
                qw += [tokenizer.decode(i.item()).strip() for i in idx][0].capitalize()
            val = val / num_sub_tokens_label
            maxi = maxi + val
            print(qw)
            print("*****")
        tot_loss +=maxi
        maxi = maxi / len(batch[0])
        maxi.backward()
        # update parameters
        optimizer.step()
        loop.set_description(f'Epoch {epoch}')
        print(maxi)
        loop.set_postfix(loss=maxi.item())
    tot_loss/=len(myDs)
    print(tot_loss)
    if tot_loss < best_loss:
        best_loss = tot_loss
    model_path = 'var_runs/model_{}_{}'.format(run_int, epoch)
    torch.save(model.state_dict(), model_path)

  0%|                                                   | 0/209 [00:00<?, ?it/s]

attributeNamesToIgnore

number of mask tok 16
number of seq 4
AttributeAttributeNames
*****
newIndex

number of mask tok 8
number of seq 4
StartIndex
*****
requestForResponse

number of mask tok 9
number of seq 3
